In [3]:
import math

import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss


def prediction_simulation(
    predictor_cls,
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    df3: pd.DataFrame,
    df4: pd.DataFrame,
    df5: pd.DataFrame,
    target: pd.Series,
):
    features = pd.concat(
        [df1["data"], df2["data"], df3["data"], df4["data"], df5["data"]],
        axis=1,
    )
    features.columns = ["feature1", "feature2", "feature3", "feature4", "feature5"]

    predictor = predictor_cls("TEST2model.cbm")

    y_pred = predictor.predict(features)

    if len(y_pred) != len(target):
        raise Exception(
            f"len of generated prediction array is not equal to target len: pred - {len(y_pred)} target - {len(target)}"
        )
    score = log_loss(target, y_pred)

    if math.isnan(score):
        raise Exception("score value is nan")

    return score


class Predictor:
    def __init__(self, model_path: str):
        self.model = CatBoostClassifier()
        self.model.load_model(model_path)

    @staticmethod
    def model_name() -> str:
        return "TEST2model.cbm"  # Название файла модели

    def predict(self, features: pd.DataFrame) -> pd.Series:
        """
        Метод вызывается для генерации предсказаний.
        """
        predict = pd.Series(self.model.predict_proba(features)[:, 1], index=features.index)
        return predict


df1 = pd.DataFrame(
    {
        "ts": range(15),
        "data": [1, 3, 6, 8, 9, 0, 3, 4, 65, 57, 643, 54, 78, 12, 654],
    }
)
df2 = pd.DataFrame(
    {
        "ts": range(15),
        "data": [32, 32, 3, 24, 124, 656, 757, 43, 323, 32, 90, 123, 434, 2, 5],
    }
)
df3 = pd.DataFrame(
    {
        "ts": range(15),
        "data": [323, 43, 5, 23, 6435, 23, 533, 232, 123, 5, 8, 3, 23, 64, 75],
    }
)
df4 = pd.DataFrame(
    {
        "ts": range(15),
        "data": range(15),
    }
)
df5 = pd.DataFrame(
    {
        "ts": range(15),
        "data": [15, 25, 23, 53, 5, 5353, 343, 535, 5, 34, 34, 75, 878, 3, 40],
    }
)
target = pd.Series([0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1])

score = prediction_simulation(Predictor, df1, df2, df3, df4, df5, target)

print("score is", score)

score is 0.3954901358845248
